In [58]:
import sys  
sys.path.insert(0, '..')

from config import Config

import pandas as pd
from sqlalchemy import create_engine

db = create_engine(Config.SQLALCHEMY_DATABASE_URI)

In [59]:
mista = pd.read_sql_query(
    """
    select ockovaci_mista.id ockovaci_misto_id, ockovaci_mista.nazev, okres_id, kraj_id 
    from ockovaci_mista join okresy on ockovaci_mista.okres_id=okresy.id;
    """,
    db
)
# mista

In [60]:
mista_ids = ','.join("'" + misto + "'" for misto in mista['ockovaci_misto_id'].tolist())

In [61]:
df = pd.read_sql_query("select * from ockovani_distribuce limit 10;", db)
# df

In [64]:
prijato = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato
    from ockovani_distribuce 
    where akce = 'Příjem' and ockovaci_misto_id in ({})
    group by (ockovaci_misto_id, vyrobce);
    """.format(mista_ids, mista_ids),
    db.engine
)
# prijato

In [65]:
prijato_odjinud = pd.read_sql_query(
    """
    select cilove_ockovaci_misto_id ockovaci_misto_id, vyrobce, sum(pocet_davek) prijato_odjinud
    from ockovani_distribuce 
    where akce = 'Výdej' and cilove_ockovaci_misto_id in ({}) and ockovaci_misto_id not in({})
    group by (cilove_ockovaci_misto_id, vyrobce);
    """.format(mista_ids, mista_ids),
    db.engine
)
# prijato_odjinud

In [67]:
vydano = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pocet_davek) vydano
    from ockovani_distribuce 
    where akce = 'Výdej' and ockovaci_misto_id in ({}) and cilove_ockovaci_misto_id not in({})
    group by (ockovaci_misto_id, vyrobce);
    """.format(mista_ids, mista_ids),
    db.engine
)
# vydano

In [68]:
df = pd.read_sql_query("select * from ockovani_spotreba limit 10;", db)
# df

In [69]:
spotreba = pd.read_sql_query(
    """
    select ockovaci_misto_id, vyrobce, sum(pouzite_davky) pouzito, sum(znehodnocene_davky) znehodnoceno
    from ockovani_spotreba 
    where ockovaci_misto_id in ({})
    group by (ockovaci_misto_id, vyrobce);
    """.format(mista_ids),
    db.engine
)
# spotreba

In [70]:
ockovano = pd.read_sql_query(
    """
    select kraj_nuts_kod, sum(pocet),
        case 
            when vakcina = 'COVID-19 Vaccine AstraZeneca' then 'AstraZeneca'
            when vakcina = 'COVID-19 Vaccine Moderna' then 'Moderna'
            when vakcina = 'Comirnaty' then 'Pfizer'
        else 'ostatní' end
    from ockovani_lide
    where kraj_nuts_kod = 'CZ042'
    group by kraj_nuts_kod, vakcina
    """,
    db.engine
)
# ockovano

In [71]:
vyrobci = pd.read_sql_query(
    """
    select distinct(vyrobce)
    from ockovani_spotreba;
    """,
    db.engine
)
# vyrobci

In [72]:
mista_key = mista
mista_key['join'] = 0

vyrobci_key = vyrobci
vyrobci_key['join'] = 0

merged = mista_key.merge(vyrobci_key).drop('join', axis=1)

merged = pd.merge(merged, prijato, how="left")
merged = pd.merge(merged, prijato_odjinud, how="left")
merged = pd.merge(merged, vydano, how="left")
merged = pd.merge(merged, spotreba, how="left")

merged['prijato'] = merged['prijato'].fillna(0).astype('int')
merged['prijato_odjinud'] = merged['prijato_odjinud'].fillna(0).astype('int')
merged['vydano'] = merged['vydano'].fillna(0).astype('int')
merged['pouzito'] = merged['pouzito'].fillna(0).astype('int')
merged['znehodnoceno'] = merged['znehodnoceno'].fillna(0).astype('int')
merged['skladem'] = merged['prijato'] + merged['prijato_odjinud'] - merged['vydano'] - merged['pouzito'] - merged['znehodnoceno']

merged

,ockovaci_misto_id,nazev,okres_id,kraj_id,vyrobce,prijato,prijato_odjinud,vydano,pouzito,znehodnoceno,skladem
0,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Moderna,0,0,0,0,0,0
1,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,Pfizer,0,0,0,0,0,0
2,63c9078b-0701-4da0-821e-4eaa0d9c9968,"Krajská zdravotní a.s., - MNUL pracoviště výst...",CZ0423,CZ042,AstraZeneca,0,0,0,0,0,0
3,a91d8128-c379-456c-b04b-1f1d6aa2da76,AGE Centrum Olomouc,CZ0712,CZ071,Moderna,0,0,0,0,0,0
4,a91d8128-c379-456c-b04b-1f1d6aa2da76,AGE Centrum Olomouc,CZ0712,CZ071,Pfizer,264,0,0,252,0,12
...,...,...,...,...,...,...,...,...,...,...,...
847,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,Pfizer,36270,0,36,8694,18,27522
848,98c76ec8-f7d1-474a-927c-d4a4fb2dcd0a,ÚVN Praha - sklad,CZ0100,CZ010,AstraZeneca,0,0,0,0,0,0
849,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Moderna,0,0,0,0,0,0
850,cb0f8a24-3c54-467e-9e67-de01f424ea85,Česká Lípa očkovací místo Crystal,CZ0511,CZ051,Pfizer,0,0,0,3738,0,-3738
